In [6]:
import pandas as pd
import numpy as np
import gc

In [7]:
file_dir = '/Volumes/Data/Row_Data/AWS시간별관측정보'
file_2203_2209 = 'AWS시간별관측정보_202203-202209.xlsx'
file_2210_2301 = 'AWS시간별관측정보_202210-202301.xlsx'
file_2302_2307 = 'AWS시간별관측정보_202302-202307.csv'
file_2308_2401 = 'AWS시간별관측정보_202308-202401.csv'


url_2203_2209 = f'{file_dir}/{file_2203_2209}'
url_2210_2301 = f'{file_dir}/{file_2210_2301}'
url_2302_2307 = f'{file_dir}/{file_2302_2307}'
url_2308_2401 = f'{file_dir}/{file_2308_2401}'

url_list = [url_2203_2209, url_2210_2301, url_2302_2307, url_2308_2401]


In [8]:
columns = ['번호', '시군명', '시군코드', '지점번호', '지점명', '법정동명', '관측일자', '관측시간', 'WGS84위도',
       'WGS84경도', '고도', '풍향', '풍속', '기온', '습도', '현지기압', '해면기압', '강수감지',
       '시간누적강우량', '일누적강우량', '데이터수집일자']

columns_new = ['관측일자', '관측시간', '기온']

In [9]:
df1 = pd.read_excel(url_2203_2209)
df2 = pd.read_excel(url_2210_2301)
df2.columns=columns
df3 = pd.read_csv(url_2302_2307, encoding='euc-kr')
df4 = pd.read_csv(url_2308_2401, encoding='euc-kr')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [10]:
sum([df1.shape[0], df2.shape[0], df3.shape[0], df4.shape[0]])

2229824

In [35]:
df = pd.concat([df1[columns_new], df2[columns_new], df3[columns_new], df4[columns_new]], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229824 entries, 0 to 2229823
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   관측일자    int64  
 1   관측시간    int64  
 2   기온      float64
dtypes: float64(1), int64(2)
memory usage: 51.0 MB


In [36]:
df.기온.info()

<class 'pandas.core.series.Series'>
RangeIndex: 2229824 entries, 0 to 2229823
Series name: 기온
Non-Null Count    Dtype  
--------------    -----  
2229824 non-null  float64
dtypes: float64(1)
memory usage: 17.0 MB


In [37]:
print( df.기온.min(), df.기온.max() )

-999.0 39.3


In [38]:
df.loc[df.기온 < -23, '기온']

9         -999.0
10        -999.0
11        -999.0
12        -999.0
14        -999.0
           ...  
2211793   -996.0
2214829   -992.0
2214965   -992.0
2215093   -992.0
2215221   -992.0
Name: 기온, Length: 97131, dtype: float64

In [39]:
df.loc[df.기온 < -23, '기온'] = pd.NA
df

,관측일자,관측시간,기온
0,20220301,0,1.3
1,20220301,0,-1.7
2,20220301,0,2.5
3,20220301,0,1.1
4,20220301,0,3.1
...,...,...,...
2229819,20230801,0,24.2
2229820,20230801,0,24.9
2229821,20230801,0,26.2
2229822,20230801,0,25.4


In [40]:
print( df.기온.min(), df.기온.max() )
df.기온.info()

-22.8 39.3
<class 'pandas.core.series.Series'>
RangeIndex: 2229824 entries, 0 to 2229823
Series name: 기온
Non-Null Count    Dtype  
--------------    -----  
2132693 non-null  float64
dtypes: float64(1)
memory usage: 17.0 MB


In [45]:
df.loc[df.기온.isna()]

,관측일자,관측시간,기온


In [44]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2132693 entries, 0 to 2229823
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   관측일자    int64  
 1   관측시간    int64  
 2   기온      float64
dtypes: float64(1), int64(2)
memory usage: 65.1 MB


In [46]:
df = df.sort_values(by=['관측일자', '관측시간'], ascending=True)
df.reset_index(inplace=True)
df = df.drop(["index"], axis=1)
df

,관측일자,관측시간,기온
0,20220301,0,1.3
1,20220301,0,-1.7
2,20220301,0,2.5
3,20220301,0,1.1
4,20220301,0,3.1
...,...,...,...
2132688,20240131,23,3.3
2132689,20240131,23,0.6
2132690,20240131,23,-0.3
2132691,20240131,23,-2.4


In [47]:
del df1
del df2
del df3
del df4
gc.collect()

120

In [48]:
df['date_time'] = df.loc[ :, ['관측일자', '관측시간'] ].apply( lambda x: pd.to_datetime(  '%s %02d:00:00'%(x[0], x[1]),  format='%Y%m%d %H:%M:%S' ), axis=1) 
df

/var/folders/r5/124yntwj1n7_xfgvd8f6pzv40000gn/T/ipykernel_2400/2765408327.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['date_time'] = df.loc[ :, ['관측일자', '관측시간'] ].apply( lambda x: pd.to_datetime(  '%s %02d:00:00'%(x[0], x[1]),  format='%Y%m%d %H:%M:%S' ), axis=1)


,관측일자,관측시간,기온,date_time
0,20220301,0,1.3,2022-03-01 00:00:00
1,20220301,0,-1.7,2022-03-01 00:00:00
2,20220301,0,2.5,2022-03-01 00:00:00
3,20220301,0,1.1,2022-03-01 00:00:00
4,20220301,0,3.1,2022-03-01 00:00:00
...,...,...,...,...
2132688,20240131,23,3.3,2024-01-31 23:00:00
2132689,20240131,23,0.6,2024-01-31 23:00:00
2132690,20240131,23,-0.3,2024-01-31 23:00:00
2132691,20240131,23,-2.4,2024-01-31 23:00:00


In [56]:
df_data = df.groupby(['date_time']).agg({'관측일자':'max', '기온':'mean'})
df_data.reset_index(inplace=True)
df_data

,date_time,관측일자,기온
0,2022-03-01 00:00:00,20220301,2.474286
1,2022-03-01 01:00:00,20220301,2.652308
2,2022-03-01 02:00:00,20220301,2.974419
3,2022-03-01 03:00:00,20220301,2.895349
4,2022-03-01 04:00:00,20220301,2.535659
...,...,...,...
16411,2024-01-31 19:00:00,20240131,4.294615
16412,2024-01-31 20:00:00,20240131,3.028462
16413,2024-01-31 21:00:00,20240131,1.810769
16414,2024-01-31 22:00:00,20240131,0.812308


In [58]:
print(df_data.관측일자.min(), df_data.관측일자.max())
df_data.관측일자.unique().shape

20220301 20240131


(690,)

In [75]:
df_date_range = pd.DataFrame(pd.date_range('20220301', '20240131', freq='D'))
df_date_range.columns = ['date']
df_date_range

,date
0,2022-03-01
1,2022-03-02
2,2022-03-03
3,2022-03-04
4,2022-03-05
...,...
697,2024-01-27
698,2024-01-28
699,2024-01-29
700,2024-01-30


In [76]:
df_data.관측일자 = df_data.관측일자.map(lambda x: pd.to_datetime(  x,  format='%Y%m%d' ))
df_data

,date_time,관측일자,기온
0,2022-03-01 00:00:00,2022-03-01,2.474286
1,2022-03-01 01:00:00,2022-03-01,2.652308
2,2022-03-01 02:00:00,2022-03-01,2.974419
3,2022-03-01 03:00:00,2022-03-01,2.895349
4,2022-03-01 04:00:00,2022-03-01,2.535659
...,...,...,...
16411,2024-01-31 19:00:00,2024-01-31,4.294615
16412,2024-01-31 20:00:00,2024-01-31,3.028462
16413,2024-01-31 21:00:00,2024-01-31,1.810769
16414,2024-01-31 22:00:00,2024-01-31,0.812308


In [78]:
df_missing_date = df_date_range.loc[ ~ df_date_range.date.isin(df_data.관측일자.unique()) ]
df_missing_date

,date
160,2022-08-08
238,2022-10-25
239,2022-10-26
240,2022-10-27
241,2022-10-28
242,2022-10-29
243,2022-10-30
244,2022-10-31
245,2022-11-01
246,2022-11-02


In [137]:
df_temp = df_data.loc[df_data.관측일자 > '2022-12-31']
df_temp.index = df_temp.date_time
df_temp.drop(columns=['date_time', '관측일자'], inplace=True)
df_temp

/var/folders/r5/124yntwj1n7_xfgvd8f6pzv40000gn/T/ipykernel_2400/400261678.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.drop(columns=['date_time', '관측일자'], inplace=True)


,기온
date_time,
2023-01-01 00:00:00,-3.054032
2023-01-01 01:00:00,-3.132800
2023-01-01 02:00:00,-3.342063
2023-01-01 03:00:00,-3.532258
2023-01-01 04:00:00,-3.738889
...,...
2024-01-31 19:00:00,4.294615
2024-01-31 20:00:00,3.028462
2024-01-31 21:00:00,1.810769


In [122]:
date_range = pd.date_range('20230101', '20240201', freq='1H')[:-1]
date_range

/var/folders/r5/124yntwj1n7_xfgvd8f6pzv40000gn/T/ipykernel_2400/723623358.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range('20230101', '20240201', freq='1H')[:-1]


DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2024-01-31 14:00:00', '2024-01-31 15:00:00',
               '2024-01-31 16:00:00', '2024-01-31 17:00:00',
               '2024-01-31 18:00:00', '2024-01-31 19:00:00',
               '2024-01-31 20:00:00', '2024-01-31 21:00:00',
               '2024-01-31 22:00:00', '2024-01-31 23:00:00'],
              dtype='datetime64[ns]', length=9504, freq='h')

In [179]:
df_dummy = pd.DataFrame(data=None, columns=df_temp.columns, index = date_range)
df_dummy['date_time'] = df_dummy.index
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9504 entries, 2023-01-01 00:00:00 to 2024-01-31 23:00:00
Freq: h
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   기온         0 non-null      object        
 1   date_time  9504 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 222.8+ KB


In [180]:
df_dummy.update(df_temp, overwrite=False)
df_dummy

,기온,date_time
2023-01-01 00:00:00,-3.054032,2023-01-01 00:00:00
2023-01-01 01:00:00,-3.1328,2023-01-01 01:00:00
2023-01-01 02:00:00,-3.342063,2023-01-01 02:00:00
2023-01-01 03:00:00,-3.532258,2023-01-01 03:00:00
2023-01-01 04:00:00,-3.738889,2023-01-01 04:00:00
...,...,...
2024-01-31 19:00:00,4.294615,2024-01-31 19:00:00
2024-01-31 20:00:00,3.028462,2024-01-31 20:00:00
2024-01-31 21:00:00,1.810769,2024-01-31 21:00:00
2024-01-31 22:00:00,0.812308,2024-01-31 22:00:00


In [181]:
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9504 entries, 2023-01-01 00:00:00 to 2024-01-31 23:00:00
Freq: h
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   기온         9445 non-null   object        
 1   date_time  9504 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 222.8+ KB


In [182]:
df_dummy.ffill( inplace=True)
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9504 entries, 2023-01-01 00:00:00 to 2024-01-31 23:00:00
Freq: h
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   기온         9504 non-null   float64       
 1   date_time  9504 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1)
memory usage: 222.8 KB


/var/folders/r5/124yntwj1n7_xfgvd8f6pzv40000gn/T/ipykernel_2400/2331655139.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dummy.ffill( inplace=True)


In [183]:
df_dummy

,기온,date_time
2023-01-01 00:00:00,-3.054032,2023-01-01 00:00:00
2023-01-01 01:00:00,-3.132800,2023-01-01 01:00:00
2023-01-01 02:00:00,-3.342063,2023-01-01 02:00:00
2023-01-01 03:00:00,-3.532258,2023-01-01 03:00:00
2023-01-01 04:00:00,-3.738889,2023-01-01 04:00:00
...,...,...
2024-01-31 19:00:00,4.294615,2024-01-31 19:00:00
2024-01-31 20:00:00,3.028462,2024-01-31 20:00:00
2024-01-31 21:00:00,1.810769,2024-01-31 21:00:00
2024-01-31 22:00:00,0.812308,2024-01-31 22:00:00


In [184]:
df_dummy.기온 = df_dummy.기온.map(lambda x: round(x))
df_dummy

,기온,date_time
2023-01-01 00:00:00,-3,2023-01-01 00:00:00
2023-01-01 01:00:00,-3,2023-01-01 01:00:00
2023-01-01 02:00:00,-3,2023-01-01 02:00:00
2023-01-01 03:00:00,-4,2023-01-01 03:00:00
2023-01-01 04:00:00,-4,2023-01-01 04:00:00
...,...,...
2024-01-31 19:00:00,4,2024-01-31 19:00:00
2024-01-31 20:00:00,3,2024-01-31 20:00:00
2024-01-31 21:00:00,2,2024-01-31 21:00:00
2024-01-31 22:00:00,1,2024-01-31 22:00:00


In [186]:
df_dummy['기온'].describe()

count    9504.000000
mean       11.639625
std        11.450109
min       -19.000000
25%         2.000000
50%        12.000000
75%        22.000000
max        34.000000
Name: 기온, dtype: float64

In [189]:
filePath = f'{file_dir}/Temp_230101_240131.csv'
df_dummy.to_csv(filePath)